# Customer.IO Data Pipelines API - Authentication and Utilities

## Purpose

This notebook demonstrates how to authenticate with Customer.IO's Data Pipelines API and use the utility modules for API operations. It covers authentication management, connection testing, request validation, and core utility functions.

## Prerequisites

- Complete setup from `00_setup_and_configuration.ipynb`
- Customer.IO API key configured in Databricks secrets
- Utility modules (`utils/`) available in the Python path

## Key Topics Covered

1. **Authentication Management** - Secure API client initialization and configuration
2. **Connection Testing** - Health checks and connection validation
3. **Rate Limiting** - Understanding and managing API rate limits
4. **Request Validation** - Using Pydantic models for type-safe requests
5. **Data Transformation** - Converting between Spark DataFrames and API formats
6. **Error Handling** - Retry patterns and graceful error management
7. **Batch Operations** - Optimizing requests for bulk data operations

## Setup and Imports

In [ ]:
# Essential imports for authentication and utilities
import sys
import os
import json
import time
from datetime import datetime, timezone, timedelta
from typing import Dict, List, Optional, Any
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Databricks and Spark imports
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from delta.tables import DeltaTable

# Import validation and logging
import structlog
from pydantic import ValidationError as PydanticValidationError

# Add utils directory to Python path
sys.path.append(os.path.join(os.getcwd(), 'utils'))

# Import Customer.IO utilities
from api_client import CustomerIOClient
from authentication_manager import AuthenticationManager, AuthenticationConfig, ConnectionStatus
from validators import IdentifyRequest, TrackRequest, GroupRequest, validate_request_size, create_context
from transformers import CustomerTransformer, EventTransformer, BatchTransformer, ContextTransformer
from error_handlers import CustomerIOError, RateLimitError, ValidationError, NetworkError, retry_on_error, ErrorContext

# Initialize logger
logger = structlog.get_logger("customerio_demo")

print("SUCCESS: Authentication and utilities imports loaded")

In [ ]:
# Import Customer.IO API utilities
from utils.api_client import CustomerIOClient
from utils.validators import (
    IdentifyRequest, 
    TrackRequest, 
    GroupRequest,
    validate_request_size,
    create_context
)

print("SUCCESS: Customer.IO API utilities imported")

In [ ]:
# Import error handling utilities
from utils.error_handlers import (
    CustomerIOError,
    RateLimitError,
    ValidationError,
    NetworkError,
    retry_on_error,
    ErrorContext
)

print("SUCCESS: Error handling utilities imported")

In [ ]:
# Import validation and logging
import structlog
from pydantic import ValidationError as PydanticValidationError

# Initialize logger
logger = structlog.get_logger("customerio_demo")

print("SUCCESS: Validation and logging initialized")

In [ ]:
# Configuration from setup notebook (secure approach)
# SECURITY: Get configuration from previous notebook, not from widgets in this notebook

try:
    # Try to get configuration from previous notebook's widgets
    CUSTOMERIO_REGION = dbutils.widgets.get("customerio_region") or "us"
    DATABASE_NAME = dbutils.widgets.get("database_name") or "customerio_demo"
    CATALOG_NAME = dbutils.widgets.get("catalog_name") or "main"
    ENVIRONMENT = dbutils.widgets.get("environment") or "test"
    
    print(f"Configuration loaded from setup notebook:")
    print(f"  Region: {CUSTOMERIO_REGION}")
    print(f"  Database: {CATALOG_NAME}.{DATABASE_NAME}")
    print(f"  Environment: {ENVIRONMENT}")
    
except Exception as e:
    print(f"WARNING: Could not load configuration from setup notebook: {str(e)}")
    print("INFO: Using fallback configuration")
    CUSTOMERIO_REGION = "us"
    DATABASE_NAME = "customerio_demo"
    CATALOG_NAME = "main"
    ENVIRONMENT = "test"

# Use current database
spark.sql(f"USE {CATALOG_NAME}.{DATABASE_NAME}")
print("SUCCESS: Configuration loaded and database selected")

In [ ]:
# Load configuration from setup notebook (secure approach)
# SECURITY: Get configuration from previous notebook, not from widgets in this notebook

try:
    # Try to get configuration from previous notebook's widgets
    CUSTOMERIO_REGION = dbutils.widgets.get("customerio_region") or "us"
    DATABASE_NAME = dbutils.widgets.get("database_name") or "customerio_demo"
    CATALOG_NAME = dbutils.widgets.get("catalog_name") or "main"
    ENVIRONMENT = dbutils.widgets.get("environment") or "test"
    
    print(f"Configuration loaded from setup notebook:")
    print(f"  Region: {CUSTOMERIO_REGION}")
    print(f"  Database: {CATALOG_NAME}.{DATABASE_NAME}")
    print(f"  Environment: {ENVIRONMENT}")
    
except Exception as e:
    print(f"WARNING: Could not load configuration from setup notebook: {str(e)}")
    print("INFO: Using fallback configuration")
    CUSTOMERIO_REGION = "us"
    DATABASE_NAME = "customerio_demo"
    CATALOG_NAME = "main"
    ENVIRONMENT = "test"

In [ ]:
# Configure Spark to use the specified database
spark.sql(f"USE {CATALOG_NAME}.{DATABASE_NAME}")
print("SUCCESS: Configuration loaded and database selected")

## Configuration and Authentication Setup

In [ ]:
# Load configuration from setup notebook
try:
    CUSTOMERIO_REGION = dbutils.widgets.get("customerio_region") or "us"
    DATABASE_NAME = dbutils.widgets.get("database_name") or "customerio_demo"
    CATALOG_NAME = dbutils.widgets.get("catalog_name") or "main"
    ENVIRONMENT = dbutils.widgets.get("environment") or "test"
    
    print(f"Configuration loaded from setup notebook:")
    print(f"  Region: {CUSTOMERIO_REGION}")
    print(f"  Database: {CATALOG_NAME}.{DATABASE_NAME}")
    print(f"  Environment: {ENVIRONMENT}")
    
except Exception as e:
    print(f"WARNING: Could not load configuration from setup notebook: {str(e)}")
    print("INFO: Using fallback configuration")
    CUSTOMERIO_REGION = "us"
    DATABASE_NAME = "customerio_demo"
    CATALOG_NAME = "main"
    ENVIRONMENT = "test"

# Use current database
spark.sql(f"USE {CATALOG_NAME}.{DATABASE_NAME}")
print("SUCCESS: Database selected")

In [ ]:
## Initialize Authentication Manager

In [ ]:
# Get API key from secure storage
try:
    if ENVIRONMENT == "test":
        CUSTOMERIO_API_KEY = "test_key_demo_12345"
        print("INFO: Using test API key")
    else:
        CUSTOMERIO_API_KEY = dbutils.secrets.get("customerio", "api_key")
        print("SUCCESS: Customer.IO API key retrieved from secure storage")
except Exception as e:
    print(f"WARNING: Could not retrieve API key: {str(e)}")
    print("INFO: Using test mode")
    CUSTOMERIO_API_KEY = "test_key_demo_12345"
    ENVIRONMENT = "test"

In [ ]:
# Create authentication configuration
auth_config = AuthenticationConfig(
    api_key=CUSTOMERIO_API_KEY,
    region=CUSTOMERIO_REGION,
    environment=ENVIRONMENT,
    timeout=30,
    max_retries=3,
    retry_backoff_factor=2.0,
    enable_logging=True,
    enable_metrics=True
)

print("SUCCESS: Authentication configuration created")
print(f"  Method: {auth_config.method.value}")
print(f"  Region: {auth_config.region}")
print(f"  Environment: {auth_config.environment}")
print(f"  Timeout: {auth_config.timeout}s")
print(f"  Max retries: {auth_config.max_retries}")

In [ ]:
# Initialize authentication manager
auth_manager = AuthenticationManager(auth_config, spark)

# Establish connection
try:
    client = auth_manager.connect()
    print("SUCCESS: Connected to Customer.IO API")
    
    # Display connection details
    metrics = auth_manager.get_metrics()
    print(f"  Connection status: {metrics['connection_status']}")
    print(f"  Rate limit: {metrics['rate_limit_status']['max_requests']} requests per {metrics['rate_limit_status']['window_seconds']}s")
    
except CustomerIOError as e:
    print(f"ERROR: Failed to connect: {str(e)}")
    if ENVIRONMENT != "test":
        raise

## Data Transformation Examples

In [ ]:
# Perform comprehensive health check
print("Testing API connection with health check...")

health_result = auth_manager.health_check()

print(f"Health Check Results:")
print(f"  Status: {health_result.status.value}")
print(f"  Is Healthy: {health_result.is_healthy}")
print(f"  API Reachable: {health_result.api_reachable}")
print(f"  Auth Valid: {health_result.auth_valid}")
print(f"  Rate Limit OK: {health_result.rate_limit_ok}")

if health_result.response_time_ms:
    print(f"  Response Time: {health_result.response_time_ms:.2f}ms")

if health_result.error_message:
    print(f"  Error: {health_result.error_message}")

if ENVIRONMENT == "test":
    print("INFO: Running in test mode - some checks may be simulated")

## Request Validation Examples

The Customer.IO API requires properly formatted requests. We use Pydantic models to ensure type safety and validation.

In [ ]:
# Test 4: Invalid track request (empty event name)
print("4. Invalid Track Request (empty event name):")

try:
    invalid_track = TrackRequest(
        userId="user_12345",
        event="",  # Empty event name
        properties={"test": "value"}
    )
    print(f"SUCCESS: Request created: {invalid_track.dict()}")
except PydanticValidationError as e:
    print(f"ERROR: Expected validation error: {e}")

In [ ]:
# Example 1: Valid identify request
print("=== Testing Request Validation ===")
print("\n1. Valid Identify Request:")

try:
    valid_identify = IdentifyRequest(
        userId="user_12345",
        traits={
            "email": "user@example.com",
            "first_name": "John",
            "last_name": "Doe",
            "plan": "premium",
            "signup_date": "2024-01-15"
        },
        timestamp=datetime.now(timezone.utc)
    )
    print(f"SUCCESS: Valid request created")
    print(f"  User ID: {valid_identify.userId}")
    print(f"  Email: {valid_identify.traits['email']}")
except PydanticValidationError as e:
    print(f"ERROR: Validation error: {e}")

In [ ]:
# Example 2: Invalid identify request (missing user identification)
print("\n2. Invalid Identify Request (missing user ID):")

try:
    invalid_identify = IdentifyRequest(
        traits={"email": "user@example.com"}
    )
    print(f"SUCCESS: Request created: {invalid_identify.dict()}")
except PydanticValidationError as e:
    print(f"ERROR: Expected validation error: {e}")

In [ ]:
# Example 4: Invalid track request (empty event name)
print("\n4. Invalid Track Request (empty event name):")

try:
    invalid_track = TrackRequest(
        userId="user_12345",
        event="",  # Empty event name
        properties={"test": "value"}
    )
    print(f"SUCCESS: Request created: {invalid_track.dict()}")
except PydanticValidationError as e:
    print(f"ERROR: Expected validation error: {e}")

In [ ]:
# Display sample identify requests
for i, request in enumerate(identify_requests[:2]):  # Show first 2
    print(f"  Request {i+1}: {json.dumps(request, indent=2, default=str)}")

In [ ]:
# Transform events to track requests
track_requests = EventTransformer.spark_to_track_requests(
    df=sample_events,
    user_id_col="user_id",
    event_name_col="event_name",
    properties_cols=["properties"],  # Use the map column as properties
    timestamp_col="timestamp"
)

print(f"Generated {len(track_requests)} track requests:")

In [ ]:
# Create ecommerce events using transformers
print("=== Creating Specialized Events ===")
print("\n1. Ecommerce Events:")

# Product viewed event
product_viewed = EventTransformer.create_ecommerce_event(
    event_name="Product Viewed",
    user_id="user_demo_001",
    product_id="prod_widget_123",
    price=29.99,
    quantity=1,
    currency="USD",
    # Additional properties
    product_name="Amazing Widget",
    category="electronics",
    brand="WidgetCorp",
    sku="WDG-123-RED"
)

print("Product Viewed Event:")
print(json.dumps(product_viewed, indent=2))

# Order completed event
order_completed = EventTransformer.create_ecommerce_event(
    event_name="Order Completed",
    user_id="user_demo_001",
    order_id="order_789",
    price=89.97,  # Total order value
    currency="USD",
    # Additional properties
    products=[
        {"product_id": "prod_widget_123", "quantity": 3, "price": 29.99}
    ],
    payment_method="credit_card",
    shipping_method="standard"
)

print("\nOrder Completed Event:")
print(json.dumps(order_completed, indent=2))

In [ ]:
# Create mobile app events
print("\n2. Mobile App Events:")

# Application opened event
app_opened = EventTransformer.create_mobile_app_event(
    event_name="Application Opened",
    user_id="user_mobile_001",
    app_version="2.1.0",
    os_version="iOS 17.2",
    device_model="iPhone 15 Pro",
    # Additional properties
    session_id="session_abc123",
    from_push_notification=False,
    app_build="2100"
)

print("Application Opened Event:")
print(json.dumps(app_opened, indent=2))

# Screen viewed event
screen_viewed = EventTransformer.create_mobile_app_event(
    event_name="Screen Viewed",
    user_id="user_mobile_001",
    app_version="2.1.0",
    # Additional properties
    screen_name="Product Details",
    screen_category="ecommerce",
    product_id="prod_widget_123"
)

print("\nScreen Viewed Event:")
print(json.dumps(screen_viewed, indent=2))

## Context Creation Examples

In [ ]:
# Create context objects for different platforms
print("=== Context Creation Examples ===")
print("\n1. Web Context:")

web_context = ContextTransformer.create_web_context(
    ip="192.168.1.100",
    user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36",
    url="https://example.com/products/widget-123",
    referrer="https://google.com/search",
    locale="en-US",
    timezone="America/New_York"
)

print(json.dumps(web_context, indent=2))

print("\n2. Mobile Context:")

mobile_context = ContextTransformer.create_mobile_context(
    app_name="CustomerIO Demo App",
    app_version="2.1.0",
    os_name="iOS",
    os_version="17.2",
    device_model="iPhone 15 Pro",
    device_id="device_12345",
    locale="en-US",
    timezone="America/Los_Angeles"
)

print(json.dumps(mobile_context, indent=2))

## Batch Operations and Size Optimization

In [ ]:
# Create batch requests and optimize sizes
print("=== Batch Operations ===")
print("\n1. Creating batch requests:")

# Create multiple requests
sample_identify = [
    {
        "userId": f"user_batch_{i}",
        "traits": {
            "email": f"user{i}@example.com",
            "plan": "premium" if i % 2 == 0 else "basic",
            "signup_date": "2024-01-15"
        }
    }
    for i in range(5)
]

sample_track = [
    {
        "userId": f"user_batch_{i}",
        "event": "Page Viewed",
        "properties": {
            "page_name": "Home",
            "url": f"https://example.com/page-{i}"
        }
    }
    for i in range(3)
]

# Create batch request
batch_requests = BatchTransformer.create_batch_request(
    identify_requests=sample_identify,
    track_requests=sample_track,
    max_batch_size=10
)

print(f"Created {len(batch_requests)} batch(es)")
for i, batch in enumerate(batch_requests):
    batch_size = BatchTransformer.estimate_batch_size(batch["batch"])
    print(f"  Batch {i+1}: {len(batch['batch'])} requests, ~{batch_size} bytes")

# Show first batch structure
if batch_requests:
    print(f"\nFirst batch preview (first 2 requests):")
    preview_batch = {"batch": batch_requests[0]["batch"][:2]}
    print(json.dumps(preview_batch, indent=2, default=str))

In [ ]:
# Demonstrate batch size optimization
print("\n2. Batch size optimization:")

# Create requests with varying sizes
large_requests = []
for i in range(20):
    # Create request with large properties to test size limits
    large_request = {
        "userId": f"user_large_{i}",
        "event": "Data Import",
        "properties": {
            "large_data": "x" * 1000,  # 1KB of data per request
            "import_id": f"import_{i}",
            "batch_number": i // 5
        }
    }
    large_requests.append(large_request)

# Optimize batch sizes to stay within limits
optimized_batches = BatchTransformer.optimize_batch_sizes(
    requests=large_requests,
    max_size_bytes=10 * 1024  # 10KB limit for demo (normally 500KB)
)

print(f"Optimized into {len(optimized_batches)} batches:")
for i, batch in enumerate(optimized_batches):
    batch_size = BatchTransformer.estimate_batch_size(batch)
    print(f"  Optimized Batch {i+1}: {len(batch)} requests, {batch_size} bytes")

## Error Handling Demonstrations

In [ ]:
# Demonstrate error handling patterns
print("=== Error Handling Demonstrations ===")
print("\n1. Error Context Manager:")

# Example of graceful error handling
with ErrorContext(
    operation_name="demo_api_call",
    logger=logger,
    raise_on_error=False,
    default_return={"status": "failed", "message": "Operation failed gracefully"}
) as error_ctx:
    # Simulate an operation that might fail
    if ENVIRONMENT == "test":
        # Simulate a controlled error for demonstration
        raise CustomerIOError("Simulated API error for demonstration")
    else:
        print("Real API call would happen here")

# Check if error occurred and get default result
if error_ctx.error:
    result = error_ctx.get_result()
    print(f"Error handled gracefully: {result}")
else:
    print("Operation completed successfully")

In [ ]:
# Demonstrate retry decorator
print("\n2. Retry Decorator Example:")

@retry_on_error(max_retries=2, backoff_factor=1.5)
def demo_api_call_with_retry(should_fail: bool = False):
    """Demo function that can be configured to fail for testing."""
    print(f"  Attempting API call (fail={should_fail})...")
    
    if should_fail:
        # Simulate different types of failures
        import random
        error_type = random.choice(["network", "server", "rate_limit"])
        
        if error_type == "network":
            raise NetworkError("Simulated network error")
        elif error_type == "server":
            raise CustomerIOError("Simulated server error", status_code=500)
        else:
            raise RateLimitError("Simulated rate limit error", retry_after=1)
    
    return {"status": "success", "message": "API call completed"}

# Test successful call
try:
    result = demo_api_call_with_retry(should_fail=False)
    print(f"SUCCESS: Success: {result}")
except Exception as e:
    print(f"ERROR: Failed: {str(e)}")

# Test failing call (will retry)
try:
    result = demo_api_call_with_retry(should_fail=True)
    print(f"SUCCESS: Success after retries: {result}")
except Exception as e:
    print(f"ERROR: Failed after retries: {str(e)}")

## Rate Limiting Demonstration

In [ ]:
# Demonstrate rate limiting behavior
print("=== Rate Limiting Demonstration ===")

# Check current rate limit status
print(f"Rate limit configuration:")
print(f"  Max requests: {client.rate_limit.max_requests}")
print(f"  Window: {client.rate_limit.window_seconds} seconds")
print(f"  Current requests: {client.rate_limit.current_requests}")
print(f"  Window start: {client.rate_limit.window_start}")

# Simulate rate limit checking
print(f"\nRate limit status:")
print(f"  Can make request: {client.rate_limit.can_make_request()}")
print(f"  Time until reset: {client.rate_limit.time_until_reset():.2f} seconds")

# Simulate some requests to show rate limiting in action
print(f"\nSimulating request tracking:")
for i in range(5):
    if client.rate_limit.can_make_request():
        client.rate_limit.record_request()
        print(f"  Request {i+1}: SUCCESS: Allowed (total: {client.rate_limit.current_requests})")
    else:
        print(f"  Request {i+1}: ERROR: Rate limited")
        break

## Request Size Validation

In [ ]:
# Demonstrate request size validation
print("=== Request Size Validation ===")

# Test normal-sized request
normal_request = {
    "userId": "user_123",
    "event": "Page Viewed",
    "properties": {
        "page_name": "Home",
        "url": "https://example.com"
    }
}

is_valid_size = validate_request_size(normal_request)
print(f"Normal request size valid: {is_valid_size}")
print(f"  Request size: ~{len(json.dumps(normal_request).encode())} bytes")

# Test oversized request
oversized_request = {
    "userId": "user_123",
    "event": "Large Data Import",
    "properties": {
        "large_payload": "x" * (33 * 1024),  # 33KB - exceeds 32KB limit
        "import_id": "large_import_001"
    }
}

is_oversized_valid = validate_request_size(oversized_request)
print(f"\nOversized request size valid: {is_oversized_valid}")
print(f"  Request size: ~{len(json.dumps(oversized_request).encode())} bytes")
print(f"  Limit: {32 * 1024} bytes (32KB)")

## Clean Up and Summary

In [ ]:
# Clean up resources
print("=== Clean Up ===")

# Disconnect authentication manager
auth_manager.disconnect()
print("SUCCESS: API connection closed")

# Display final metrics
final_metrics = auth_manager.get_metrics()
print(f"\nSession Metrics:")
print(f"  Total requests: {final_metrics['total_requests']}")
print(f"  Success rate: {final_metrics['success_rate']:.1f}%")
print(f"  Average response time: {final_metrics['average_response_time']:.2f}ms")

print("\nCOMPLETED: Authentication and utilities demonstration complete!")